In [1]:
from google.colab import drive
drive.mount('/content/drive/')



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install beautifultable

In [3]:
from IPython import get_ipython
get_ipython().magic('reset -sf')
import pandas as pd
from graphviz import Digraph
import pydot
import networkx as nx
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy
import math
import cv2
import os
os.chdir('drive/MyDrive/Colab Notebooks')

In [4]:
import numpy as np
from graphviz import Digraph

class decisionTree:
    def __init__(self, features_name_list = [], label_name = '', dataframe = pd.DataFrame() , algorithm = ''):
        self.featuresList = features_name_list
        self.labelName = label_name
        self.data = dataframe
        self.alg = algorithm
        self.graph = pydot.Dot(graph_type='digraph')
        self.counter = 1
        self.counter2 = 1
        self.edges = []
        self.flag = False
        self.label_value = np.unique(self.data[self.labelName])
        
    
    def Entropy(self, feature_name = '', dataframe =  pd.DataFrame()):                               
        d = dataframe[feature_name]
        unique = np.unique(d[:])
        entropy = 0
        for value in unique:
            temp = (d == value).sum()
            temp /= d.shape[0]
            entropy -= (temp * math.log2(temp))
        return entropy
      
    def Conditional_Entropy(self, feature_name = '', feature_value = '', dataframe =  pd.DataFrame()): 
        d = dataframe.loc[dataframe[feature_name] == feature_value]
        unique = np.unique(d[self.labelName])
        entropy = 0
        for value in unique:
            temp = (d[self.labelName] == value).sum()
            temp /= d[self.labelName].shape[0]
            entropy -= (temp * math.log2(temp))
        return entropy
    
    def Feature_Based_Entropy(self, feature_name = '', dataframe =  pd.DataFrame()):
        unique = np.unique(dataframe[feature_name])
        entropy = 0
        for value in unique:
            p = ((dataframe[feature_name] == value).sum())/(dataframe[feature_name].shape[0])
            entropy += p * self.Conditional_Entropy(feature_name, value, dataframe)
        return entropy
      
    def Information_Gain(self, feature_name = '', dataframe =  pd.DataFrame()):
        return (self.Entropy(self.labelName, dataframe) - self.Feature_Based_Entropy(feature_name, dataframe))
      
    def Gain_Ratio(self, feature_name = '', dataframe =  pd.DataFrame()):
        return (self.Information_Gain(feature_name, dataframe) / self.Entropy(feature_name, dataframe))
      
    def Create_Tree(self, knowledge_features = [], knowledge_values = [], root_label = ''):
      d = self.data.copy()
      self.counter += 1
      des = ''
      temp = self.featuresList.copy()
      for i in range(0, len(knowledge_features)):
          temp.remove(knowledge_features[i]) 
          d = d.loc[d[knowledge_features[i]] == knowledge_values[i]]

      if self.Entropy(self.labelName, d) == 0 or len(temp) == 0:
          if len(knowledge_values) > 0:
              edge = pydot.Edge(root_label, str(np.unique(d[self.labelName])[0]) + ' : ' + str(knowledge_values[-1])
                              + ' : ' + str(self.counter) , label = str(knowledge_values[-1]))
              self.graph.add_edge(edge)
              self.edges.append([root_label , str(np.unique(d[self.labelName])[0]) + ' : ' + str(knowledge_values[-1])
                            + ' : '  + str(self.counter) ,  str(knowledge_values[-1])])
          else:
              edge = pydot.Edge(root_label, str(np.unique(d[self.labelName])[0]) + ' : ' + str(self.counter))
              self.graph.add_edge(edge)
              self.edges.append([root_label , str(np.unique(d[self.labelName])[0]) + str(self.counter), 'root'])



      else:
          maximum_confidence = -1
          maximum_confidence_index = -1
          for i in range(0, len(temp)):
              if self.alg == 'ID3':
                  confidence_value = self.Information_Gain(temp[i], d)
              elif self.alg == 'C4.5':
                  confidence_value = self.Gain_Ratio(temp[i], d)
              if confidence_value >= maximum_confidence:
                  maximum_confidence = confidence_value
                  maximum_confidence_index = i

          if len(knowledge_values) > 0:     
              des = temp[maximum_confidence_index]  + ' : ' + str(knowledge_values[-1]) + ' : ' + str(self.counter)
              edge = pydot.Edge(root_label, des, label = str(knowledge_values[-1]))
              self.edges.append([root_label ,  des , str(knowledge_values[-1])])
          else:
              des = temp[maximum_confidence_index] + ' : ' + str(self.counter)
              edge = pydot.Edge(root_label, des)
              self.edges.append([root_label, des, 'root'])

          self.graph.add_edge(edge)

          feature_value = knowledge_features
          feature_value.append(temp[maximum_confidence_index])
          label_value = knowledge_values
          label_value.append('')

          for i in np.unique(d[temp[maximum_confidence_index]]):
            label_value[len(label_value)-1] = i
            self.Create_Tree(feature_value.copy() , label_value.copy(), des)
      
                
          
    def evaluate_binary_class(self, test_data = pd.DataFrame(), pos_value = '' , neg_value = ''):
        TP = 0 
        TN = 0
        FP = 0
        FN = 0
        OW = 0
        
        for i in range (0, test_data.shape[0]):
            flag = True
            current_state = self.edges[0][1]
            current_feature = current_state.split(' : ')[0]
            d = test_data.iloc[i:i+1,:]
           
            while (current_feature != pos_value and current_feature != neg_value and flag != False):
            
                
                column = [current_feature]
                value = pd.DataFrame(d, columns=column).values[0][0]
                flag = False
                for edge in self.edges:
                    if edge[0] == current_state and edge[2] == value:
                        current_state = edge[1]
                        current_feature = current_state.split(' : ')[0]
                        flag = True
            y_pred.append(current_feature)           
            if current_feature == pos_value and d[self.labelName].values == pos_value:
                TP += 1
            elif current_feature == pos_value and d[self.labelName].values == neg_value:
                FP += 1
            elif current_feature == neg_value and d[self.labelName].values == neg_value:
                TN += 1
            elif current_feature == neg_value and d[self.labelName].values == pos_value:
                FN += 1
            else:
                OW += 1
        return str(TP) + ',' + str(FP) + ',' + str(TN) + ',' + str(FN) + ',' + str(OW)

    def ypred_output(self, test_data = pd.DataFrame(), pos_value = '' , neg_value = ''):
        TP = 0 
        TN = 0
        FP = 0
        FN = 0
        OW = 0
        qw=[]
        for i in range (0, test_data.shape[0]):
            flag = True
            current_state = self.edges[0][1]
            current_feature = current_state.split(' : ')[0]
            d = test_data.iloc[i:i+1,:]
           
            while (current_feature != pos_value and current_feature != neg_value and flag != False):
            
                
                column = [current_feature]
                value = pd.DataFrame(d, columns=column).values[0][0]
                flag = False
                for edge in self.edges:
                    if edge[0] == current_state and edge[2] == value:
                        current_state = edge[1]
                        current_feature = current_state.split(' : ')[0]
                        flag = True
            qw.append(current_feature)           
            if current_feature == pos_value and d[self.labelName].values == pos_value:
                TP += 1
            elif current_feature == pos_value and d[self.labelName].values == neg_value:
                FP += 1
            elif current_feature == neg_value and d[self.labelName].values == neg_value:
                TN += 1
            elif current_feature == neg_value and d[self.labelName].values == pos_value:
                FN += 1
            else:
                OW += 1
                
        return qw
           
                
    def show_confusion_matrix(self, test_data = pd.DataFrame(), pos_value = '', neg_value = ''):
        ls = self.evaluate_binary_class(test_data, pos_value, neg_value).split(',')
        from beautifultable import BeautifulTable
        table = BeautifulTable()
        table.column_headers = ["label", "classifier output positive", "classifier output negative"]
        table.append_row(["Actual Positive", str(ls[0]), str(ls[3])])
        table.append_row(["Actual Negative", str(ls[1]), str(ls[2])])
        print(table)


    def show_complete_data(self, test_data = pd.DataFrame(), pos_value = '', neg_value = ''):
        ls = self.evaluate_binary_class(test_data, pos_value, neg_value).split(',')
        return ls
         
      
    def recall_binary_class(self, test_data = pd.DataFrame(), pos_value = '', neg_value = ''):
        ls = self.evaluate_binary_class(test_data, pos_value, neg_value).split(',')
        TP = int(ls[0])
        TN = int(ls[2])
        FP = int(ls[1])
        FN = int(ls[3])
        OW = int(ls[4])
        recall = TP / (TP + FN + OW)
        return recall

    def precision_binary_class(self, test_data = pd.DataFrame(), pos_value = '', neg_value = ''):
        ls = self.evaluate_binary_class(test_data, pos_value, neg_value).split(',')
        TP = int(ls[0])
        TN = int(ls[2])
        FP = int(ls[1])
        FN = int(ls[3])
        precision = TP / (TP + FP)
        return precision
      
    def npv_binary_class(self, test_data = pd.DataFrame(), pos_value = '', neg_value = ''):
        ls = self.evaluate_binary_class(test_data, pos_value, neg_value).split(',')
        TP = int(ls[0])
        TN = int(ls[2])
        FP = int(ls[1])
        FN = int(ls[3])
        OW = int(ls[4])
        npv = TN / (TN + FN + OW)
        return npv
      
    def spec_binary_class(self, test_data = pd.DataFrame(), pos_value = '', neg_value = ''):
        ls = self.evaluate_binary_class(test_data, pos_value, neg_value).split(',')
        TP = int(ls[0])
        TN = int(ls[2])
        FP = int(ls[1])
        FN = int(ls[3])
        spec = TP / (TP + FP)
        return spec
      
    def accuracy_binary_class(self, test_data = pd.DataFrame(), pos_value = '', neg_value = ''):
        ls = self.evaluate_binary_class(test_data, pos_value, neg_value).split(',')
        TP = int(ls[0])
        TN = int(ls[2])
        FP = int(ls[1])
        FN = int(ls[3])
        OW = int(ls[4])
        accuracy = ((TP + TN) / (TP + FP + TN + FN + OW))
        return accuracy

In [5]:
import pandas as pd
data_1 = pd.read_csv('mushrooms.csv')
from sklearn.utils import shuffle
data_1 = shuffle(data_1)
data_nolabels=data_1.drop(['class'],axis=1)
sizeo=data_1.shape[0]
data_1_train = data_1.iloc[0:int(sizeo*2/3),0:23]
data_1_test = data_1.iloc[int(sizeo*2/3):,0:23]
train=data_1.iloc[:,1:]
test=list(data_1_test.iloc[:,0])
for i in range(len(test)):
  if(test[i]=='p'):
    test[i]=1
  else:
    test[i]=0
shap=data_1_test.shape[0]
data_1.head()
y_pred=[]

In [6]:
output_label_name = 'class'
file = open('mushrooms.csv', 'r') 
labels=(file.readline().split(','))
labels.remove(output_label_name)
labels.remove('habitat\n')
labels.append('habitat')
features_name_list=labels
print(features_name_list)



['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']


In [7]:
dt = decisionTree(features_name_list, output_label_name , data_1_train, 'C4.5')
dt.Create_Tree([], [], 'Start')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars


In [8]:
dt.show_confusion_matrix(data_1_test, 'p', 'e')
recall=dt.recall_binary_class(data_1_test, 'p', 'e')* 100
print('Recall = ' + str(recall)+'%')
precision=dt.precision_binary_class(data_1_test, 'p', 'e') * 100
print('Precision (Positive Predictive Value) = ' + str(precision)+'%')
print('Negative Predictive Value = ' + str(dt.npv_binary_class(data_1_test, 'p', 'e') * 100)+'%')
print('Specificity = ' + str(dt.spec_binary_class(data_1_test, 'p', 'e') * 100)+'%')
print('Accuracy = ' + str(dt.accuracy_binary_class(data_1_test, 'p', 'e') * 100)+'%')
y_pred=y_pred[:shap]
f1=2*recall*precision/(recall+precision)
print(f'F1-score is {f1}')
w=dt.show_confusion_matrix(data_1_test, 'p', 'e')


/usr/local/lib/python3.7/dist-packages/beautifultable/utils.py:125: FutureWarning: 'BeautifulTable.column_headers' has been deprecated in 'v1.0.0' and will be removed in 'v1.2.0'. Use 'BTColumnCollection.header' instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/beautifultable/utils.py:125: FutureWarning: 'BeautifulTable.append_row' has been deprecated in 'v1.0.0' and will be removed in 'v1.2.0'. Use 'BTRowCollection.append' instead.
  warnings.warn(message, FutureWarning)


+-----------------+----------------------------+----------------------------+
|      label      | classifier output positive | classifier output negative |
+-----------------+----------------------------+----------------------------+
| Actual Positive |            1237            |             42             |
+-----------------+----------------------------+----------------------------+
| Actual Negative |             0              |            1429            |
+-----------------+----------------------------+----------------------------+
Recall = 96.71618451915559%
Precision (Positive Predictive Value) = 100.0%
Negative Predictive Value = 97.14479945615227%
Specificity = 100.0%
Accuracy = 98.44903988183161%
F1-score is 98.33068362480127
+-----------------+----------------------------+----------------------------+
|      label      | classifier output positive | classifier output negative |
+-----------------+----------------------------+----------------------------+
| Actual Positive

In [9]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
import plotly.express as px
import plotly.graph_objects as go


for n in range(3,11):
  record_number = data_1.shape[0]
  divide_result = math.floor(record_number / n)

  partition_list = []
  
  for Fold in range(0, n):
      partition_list.append([Fold * divide_result, Fold * divide_result + divide_result])
  qw=[]
  y2_pred=[]
  val=1
  for Fold in range(0, n-1):
      d = data_1
      d = d.drop(d.index[[partition_list[Fold][0], partition_list[Fold][1]]])
      dtree = decisionTree(features_name_list, output_label_name, d, 'C4.5')
      dtree.Create_Tree([], [], 'Start')
      validation = data_1.iloc[partition_list[Fold][0]:partition_list[Fold][1],:].copy()
      qw.append(dtree.accuracy_binary_class(validation, 'p', 'e') * 100)
      while(val>0):
        qwer=dtree.ypred_output(validation,'p','e')
        val-=1
        for i in range(len(qwer)):
          if(qwer[i]=='p'):
            qwer[i]=1
          else:
            qwer[i]=0
        test1=list(validation.iloc[:,0])
        for i in range(len(test1)):
          if(test1[i]=='p'):
            test1[i]=1
          else:
            test1[i]=0
        if(n==7):
          fig = go.Figure()
          false_positive_rate,true_positive_rate,threshold = roc_curve(test1,qwer)

          fig.add_trace(go.Scatter(x=false_positive_rate, y=true_positive_rate,
                        mode='lines',
                        name='ROC'))
          fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], name='',
                            line=dict(color='purple', width=2, dash='dash')))

          fig.update_layout( title="Receiver Operating Characterstic Curve",title_x = 0.5,xaxis_title="False positive rate", yaxis_title="True positive rate")

          fig.show()
        


  qwe=sum(qw)/len(qw)
  print(f'Avg accuracy for {n}-fold cross validation is {qwe}')

      


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: RuntimeWarning:

invalid value encountered in double_scalars



Avg accuracy for 3-fold cross validation is 98.55982274741507
Avg accuracy for 4-fold cross validation is 98.65419333661578
Avg accuracy for 5-fold cross validation is 98.64532019704433
Avg accuracy for 6-fold cross validation is 98.61152141802067


Avg accuracy for 7-fold cross validation is 98.60632183908046
Avg accuracy for 8-fold cross validation is 98.60661505981703
Avg accuracy for 9-fold cross validation is 98.60033259423503
Avg accuracy for 10-fold cross validation is 98.53585112205802
